# Anvil Real Estate (Demo)*
(c) 2018 David Williams. All Rights Reserved

**What**: A data science app that evaluates properties to identify parcels with a likelihood (0-1, where 1=100%) of profitability (unprofitable vs profitable when sold). 

In this demo, the app evaluates Hawaii County, HI. The code derives a machine learning model from past sales history and broader economic conditions. Then the model is used to assign a likelihood of sales outcomes to each parcel in the current year using up-to-date economic measures and most recent sales figures. These predictions can be used to symbolize real estate on a map and evaluate investment opportunities at a glance.

**Use**: Allows a generic investor with no prior knowledge of a market to quickly determine a property's *likely* economic status, lowering the information barrier of entry to a real estate market.

**Steps**

When selecting "Run All Cells" above, the average run time will take approximately 20 min. Results will be displayed in an ArcGIS map widget at the bottom of the notebook. 

1. <a href='#Step-1:-Get-the-Data'>Acquire data</a> 

2. <a href='#Step-2:-Inspect-Data-Distributions'>Inspect Data Distributions</a>

3. <a href='#Step-3:-Create-Machine-Learning-Model-(Supervised)'>Create machine learning (ML) model</a>

4. <a href='#Run-ML-Model-and-Score'>Run ML model backtest and make current status predictions</a>

5. <a href='#Step-5:-Visualize-Results-with-ArcGIS'>Display properties in a map, symbolized by likelihood of profitability</a>



\* This software is for demo purposes. Signifigant portions of data and other programming routines have been omitted from this demo. Prediction results from the full application can vary. If you would like to know more or have a feature request, please write to <a href='mailto:info@improvz.com'>info@improvz.com</a> and include the subject line 'Anvil Data App'.

### Dependencies

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import sys, os, re, datetime, time

import requests
from bs4 import BeautifulSoup as soup

import numpy as np
import pandas as pd
import sqlite3 

import shapely
import geopandas as gpd

from modules.anvil_mods import *

## Step 1: Get the Data

SALE HISTORY

In [ ]:
# connect to db. Note: acquisition routines for this data not provided with this demo app.
dbcnx = sqlite3.connect('data/hi_county_parcels.db')
# load the data
hi_sales = pd.read_sql('SELECT * FROM sample_data;', con=dbcnx).drop(['index'], axis=1)
# close db connection 
dbcnx.close()

In [ ]:
# set the parcel num to int
hi_sales['parcel'] = hi_sales['parcel'].astype(int) 
# convert all dates to dt
hi_sales['Sale Date'] = pd.to_datetime(hi_sales['Sale Date'], errors='coerce')
# convert sale amt to float
hi_sales['Sale Amount'] = hi_sales['Sale Amount'].str.replace('[$,]','').astype(float)
# use only sales where amt > 0
hi_sales = hi_sales[hi_sales['Sale Amount'] > 0]

In [ ]:
# get the first and last dates of all sale dates
start_date = hi_sales[hi_sales['Sale Date'].dt.year > 1900].min()['Sale Date']
end_date = hi_sales[hi_sales['Sale Date'].dt.year > 1900].max()['Sale Date']
# mask away sale dates for which start date <= x <= end date  
mask = (hi_sales['Sale Date'] >= start_date) & (hi_sales['Sale Date'] <= end_date)
hi_sales = hi_sales[mask]
# free memory
del(mask)

In [ ]:
# drop duplicate sales, identified by date and parcel
hi_sales.drop_duplicates(subset=['Sale Date', 'parcel'], inplace=True)
# remove records without a sale amount
hi_sales = hi_sales[~hi_sales['Sale Amount'].isin([np.NaN])]

In [ ]:
# create an index with all parcels that have repeat sales history
test = hi_sales['parcel'].value_counts().gt(1)
# filter out only True values
idx = test[test==True].index
# create a container with all repeat sales
parcels_w_hist = hi_sales[hi_sales['parcel'].isin(idx)]
# free memory
del(hi_sales)

In [ ]:
# set a column to indicate profitability
parcels_w_hist['Profitable Sale'] = 0

# determine the sale profitability status by:
# sort parcels and dates so most recent items are first for comparison
parcels_w_hist.sort_values(['parcel','Sale Date'], ascending=False, inplace=True)
# create a tf table
tf = parcels_w_hist['Sale Amount'] > parcels_w_hist['Sale Amount'].shift(-1)

In [ ]:
# create boolean index for last items

# first make an empty list
last_parcels = []
# get list of unique parcels
unique_parcels = parcels_w_hist['parcel'].unique()

# for each parcel
for parcel in unique_parcels: 
    # isolate parcel sale history within a separate df
    idx = parcels_w_hist[parcels_w_hist['parcel'] == parcel].iloc[-1].name
    # append the parcel index
    last_parcels.append(idx)

# remove items from tf where item is last in index
mask_tf = (~tf.index.isin(last_parcels)) & (tf == True)
# select the parcels using mask and set profitable sale to 1
parcels_w_hist.loc[mask_tf, 'Profitable Sale'] = 1
# free memory
del(unique_parcels)

#### County Data

In [ ]:
# data products
HI_geo_stem = 'https://opendata.arcgis.com/datasets/'
HI_parcels_json = '1eb5fa03038d49cba930096ea67194e0_5'
HI_zoning_json = 'e81bcaf8da9a4140a09aa922ef918ea4_2' 

PARCEL DATA AS GEOJSON

In [ ]:
# create download link
parcels_link = ''.join([HI_geo_stem, HI_parcels_json, '.geojson'])
# get json
raw_parcels_json = requests.get(parcels_link).json()
# load into geopandas
hi_parcels = convertGeoJsonGeometry(raw_parcels_json)

In [ ]:
# separate parcel names
hi_parcels['name'] = hi_parcels.qpub_link.str.extract('(?P<parcel>\d{10,25})').astype(int)
# extract desired cols
columns = ['geometry', 'bldgexempt', 'bldgvalue', 'gisacres', 
           'homeowner', 'landexempt', 'landvalue', 'name', 'pittcode', 
           'st_areashape', 'taxacres']
hi_parcels = hi_parcels[columns]

In [ ]:
# free memory
del(parcels_link, raw_parcels_json, columns)

ZONING DATA AS GEOJSON

In [ ]:
# zoning 
zoning_link = ''.join([HI_geo_stem, HI_zoning_json, '.geojson'])
# get json
raw_zones_json = requests.get(zoning_link).json()
# get only desired cols
columns = ['zone', 'zoning_id','district']
# load into geopandas
hi_zones_gpd = convertSpecGeoJsonGeometry(raw_zones_json, columns)
# remove all roads 
hi_zones_gpd = hi_zones_gpd[~hi_zones_gpd['zone'].str.contains('road')] 
# remove zone column
hi_zones_gpd.drop(['zone'], axis=1, inplace=True)
# free memory
del(raw_zones_json, columns, zoning_link)

In [ ]:
# spatially join features: parcels with zones
hi_parcels_w_zones = spatialJoinFeatures(hi_parcels, hi_zones_gpd) # !! about 10min runtime on single core !!
# clean up cols
hi_parcels_w_zones.columns = hi_parcels_w_zones.columns.str.replace('properties.','')
# drop those parcels that have more than one entry
# names = hi_parcels_w_zones['name'].value_counts().gt(1).index
hi_parcels_w_zones.drop_duplicates(subset='name', keep=False, inplace=True)
# free memory
del(hi_parcels, hi_zones_gpd)

In [ ]:
# format the names so they can be merged with hi_sales
# change the name column to be matched wth hi_sales on a merge
hi_parcels_w_zones.rename({'name':'parcel'}, inplace=True, axis=1)
# format parcel numbers as ints so they match hi_sales
hi_parcels_w_zones['parcel'] = hi_parcels_w_zones['parcel'].astype(int)

#### Clean & Sort HI County Tax Authority Data

In [ ]:
# merge hi_parcels_w_zones with sales 
parcels_w_sales = pd.merge(hi_parcels_w_zones, parcels_w_hist, how='inner', on='parcel')

In [ ]:
# free memory
del(hi_parcels_w_zones, parcels_w_hist)

SET SALE DATE INTERVALS FOR EACH PARCEL

In [ ]:
# reset the index before determing sale date intervals
parcels_w_sales.reset_index(drop=True, inplace=True)
# set intervals
parcels_w_sales['Sale Intvl'] = getDateIntvlByParcel(parcels_w_sales[['parcel', 'Sale Date']])

ADD OTHER LAND & BUILDING INFO

In [ ]:
# set new connection
dbcnx= sqlite3.connect('data/hi_county_parcels.db')
# get parcel data
parceldat = pd.read_sql('SELECT * FROM parceldat', con=dbcnx)
dbcnx.close()

In [ ]:
# convert all digits to float or int
parceldat['land_area'] = parceldat['land_area'].str.replace('[,]', '').astype(float)
parceldat['mkt_bld_val'] = parceldat['mkt_bld_val'].str.replace('[,$]', '').astype(float)
parceldat['mkt_land_val'] = parceldat['mkt_land_val'].str.replace('[,$]', '').astype(float)
parceldat['tot_tax_val'] = parceldat['tot_tax_val'].str.replace('[,$]', '').astype(float)
parceldat['parcel'] = parceldat['parcel'].astype(int) 

In [ ]:
# merge parcels_w_hist with parceldat
parcels_w_sales = parcels_w_sales.merge(parceldat, on='parcel')
# free memory
del(parceldat)

#### Attach State-wide Data

BUFFER DISTANCE FOR EACH PARCEL TO GET SPATIAL INTERSECTION WITH FEATURES

In [ ]:
# converting degrees to miles for buffer distance
deg_to_mi = .001 * 14.4569170002

In [ ]:
# remove parcels w/o a geometry
parcels_w_sales = parcels_w_sales[~parcels_w_sales['geometry'].isnull()]
# create a 1mi radius around each property.. * n | n=desired miles.
parcels_w_sales['buff_dist'] = gpd.GeoDataFrame(parcels_w_sales.buffer(deg_to_mi))
# set the new geometry for calculations
parcels_w_sales = parcels_w_sales.set_geometry('buff_dist')

SPATIAL INTERSECTION: ROADS & OTHER INFRASTRUCTURE

In [ ]:
# roads supplied by Hawaii GIS dept.
hi_roads_all = '636620e50cce4b44820c545eb3195a31_6' # all roads
hi_roads_st = '1598e6509d574dceafb939a945a2075b_11' # state
hi_roads_maj = 'c32f8d888edc4f1fa428973f785f9eee_8' # major 

In [ ]:
# make road links
hi_roads_all_link = ''.join([HI_geo_stem, hi_roads_all, '.geojson'])
hi_roads_st_link = ''.join([HI_geo_stem, hi_roads_st, '.geojson'])
hi_roads_maj_link = ''.join([HI_geo_stem, hi_roads_maj, '.geojson'])

# acquire all roads data 
hi_roads_all_json = requests.get(hi_roads_all_link).json()
# convert to geopandas
hi_roads_all_gpd = convertSpecGeoJsonGeometry(hi_roads_all_json, ['fullname', 'featureuid'])
# rename col
hi_roads_all_gpd.rename({'fullname':'road'}, inplace=True, axis=1)

# acquire state roads
hi_roads_st_json = requests.get(hi_roads_st_link).json()
# convert to geopandas
hi_roads_st_gpd = convertSpecGeoJsonGeometry(hi_roads_st_json, ['route_name', 'island', 'featureuid'])
# extract hi only 
hi_roads_st_gpd = hi_roads_st_gpd[hi_roads_st_gpd['island'] == 'Hawaii'].drop(['island'], axis=1)
# rename col
hi_roads_st_gpd.rename({'route_name':'road'}, inplace=True, axis=1)

# acquire major routes
hi_roads_maj_json = requests.get(hi_roads_maj_link).json()
# convert to geopandas
hi_roads_maj_gpd = convertSpecGeoJsonGeometry(hi_roads_maj_json, ['fename', 'featureuid'])
# rename col
hi_roads_maj_gpd.rename({'fename':'road'}, inplace=True, axis=1)

# concat all roads into single record
all_roads_gpd = pd.concat([hi_roads_all_gpd, hi_roads_st_gpd, hi_roads_maj_gpd], sort=True)
# free memory
del(hi_roads_all, hi_roads_st, hi_roads_maj,
    hi_roads_all_link, hi_roads_st_link, hi_roads_maj_link,
    hi_roads_all_json, hi_roads_st_json, hi_roads_maj_json,
    hi_roads_all_gpd, hi_roads_st_gpd, hi_roads_maj_gpd,
   )

In [ ]:
# do buffer for each parcel and count number w/in proximity
parcels_w_roads = spatialJoinFeatures(parcels_w_sales[['parcel', 'buff_dist']], all_roads_gpd) #!! 10min run time
# for each parcel, create a count table and use that for lookup
# then add column for num banks within radius and apply count
parcels_w_counts = getCountForSpatialJoin(parcels_w_sales['parcel'], parcels_w_roads, 'road', 'featureuid')
# free up memory
del(all_roads_gpd, parcels_w_roads)
# add count column to parcels_w_hist for roads nearby.
parcels_w_sales['roads_nearby'] = parcels_w_sales['parcel'].apply(lambda x: parcels_w_counts.get(x)) 
# free up memory
del(parcels_w_counts)

DIGITAL ELEVATION MODEL DERIVED DATA

In [ ]:
# SOURCE: http://www.soest.hawaii.edu/coasts/data/hawaii/dem.html

# load pre-computed elevation data from CSV
dem_stats = pd.read_csv('data/hi_parcel_elev_slope_stats.csv').drop(['AREA', 'SUM', 'SUM_1','COUNT'], axis=1)
# rename cols
col_names = {
    'MIN':'ELEV_MIN',
    'MAX':'ELEV_MAX',
    'RANGE':'ELEV_RANGE',
    'MEAN':'ELEV_MEAN',
    'STD':'ELEV_STD',
    'MIN_1':'SLOPE_MIN',
    'MAX_1':'SLOPE_MAX',
    'RANGE_1':'SLOPE_RANGE',
    'MEAN_1':'SLOPE_MEAN',
    'STD_1':'SLOPE_STD',
}
dem_stats.rename(col_names, axis=1, inplace=True)
# set the dtype for parcels
dem_stats['parcel'] = dem_stats['parcel'].astype(int)
# merge dem stats with parcels_w_hist by parcel number
parcels_w_sales = pd.merge(parcels_w_sales, dem_stats, on='parcel', how='inner')
# free up memory
del(dem_stats)

SPATIAL INTERSECTION: SCHOOLS

In [ ]:
# make public and private school json links
school_link_pub = ''.join([HI_geo_stem, 'ba92e2fc0d2d4497af7289cb4ab552a7_7', '.geojson'])
school_link_priv = ''.join([HI_geo_stem, '95e39721d53c4319a5a4525866515458_8', '.geojson'])

# filter desired cols
priv_cols = ['island', 'address', 'school', 'students11', 'tuition11', 'zipcode']
pub_cols = ['district', 'address', 'city', 'esis_name', 'island', 'objectid', 'grade_from', 'grade_to', 'name', 'zip', 'type']

columns = ['name', 'island', 'geometry', 'address'] 

# get pub school json
hi_pubschool_json = requests.get(school_link_pub).json()
# convert to geopandas
hi_pubschool_gpd = convertSpecGeoJsonGeometry(hi_pubschool_json, pub_cols) 
# filter desired cols
hi_pubschool_gpd = hi_pubschool_gpd[columns]

# get priv school json
hi_privschool_json = requests.get(school_link_priv).json()
# convert to geopandas
hi_privschool_gpd = convertSpecGeoJsonGeometry(hi_privschool_json, priv_cols)
# rename the hi_privschool_gpd 'name' col to "school" 
hi_privschool_gpd.rename({'school':'name'}, inplace=True, axis=1)
# filter desired cols
hi_privschool_gpd = hi_privschool_gpd[columns]

# join the public and private school data 
hi_schools_gpd = pd.concat([hi_privschool_gpd, hi_pubschool_gpd])
# filter island to Hawaii
hi_schools_gpd = hi_schools_gpd[hi_schools_gpd['island'] == 'Hawaii']
# keep only needed cols
hi_schools_gpd = hi_schools_gpd[['geometry', 'name', 'address']]

# do buffer for each parcel and count number w/in proximity
parcels_w_schools = spatialJoinFeatures(parcels_w_sales[['parcel', 'buff_dist']], hi_schools_gpd)
# for each parcel, create a count table and use that for lookup
# then add column for num banks within radius and apply count
parcels_w_counts = getCountForSpatialJoin(parcels_w_sales['parcel'], parcels_w_schools, 'name', 'address')
# series with counts of roads within radius of each parcel

# free up memory
del(
    school_link_priv, school_link_pub, 
    priv_cols, pub_cols, columns, 
    hi_pubschool_json, hi_privschool_json, 
    hi_pubschool_gpd, hi_privschool_gpd,
    hi_schools_gpd
   )

# add the count to parcels_w_hist
parcels_w_sales['schools_nearby'] = parcels_w_sales.parcel.apply(lambda x: parcels_w_counts.get(x))

# free up memory
del(parcels_w_counts, parcels_w_schools)

#### Federal Data

In [ ]:
import quandl
from fred import Fred

In [ ]:
parcels_w_sales['Year'] = parcels_w_sales['Sale Date'].dt.year

FEDERAL RESERVE

In [ ]:
# FRED: fed funds / interbank lending rate: https://fred.stlouisfed.org/series/IR3TIB01USM156N

# yield curves: https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yield - daily yield curve
fed_funds = quandl.get("FRED/DFF", start_date=start_date, end_date=end_date) #.plot()
# group fedfunds by year and extract the sum of the differences for each year
fed_funds_mvt = getPeriodicIndexMovement(fed_funds)
# get the sale year's fed funds record using a dict to lookup the value
parcels_w_sales['Fed Funds'] = parcels_w_sales['Year'].apply(lambda x: fed_funds_mvt.get(x))
# free memory by removing unneeded records
del(fed_funds, fed_funds_mvt)

In [ ]:
# QUANDL case-shiller index
# https://www.quandl.com/data/FED/FL075035223_A-Interest-rates-and-price-indexes-owner-occupied-real-estate-S-P-Case-Shiller-national-SA-Annual-Levels-NSA

# get the case_shiller = quandl.get("FED/FL075035223_A", start_date=start_date, end_date=end_date)
case_shill_idx = formatIndicatorLikeQuandl('CSUSHPINSA')
# group the movement and get the sum of differences for each year
case_shill_mvt = getPeriodicIndexMovement(case_shill_idx)
# set the sale year's case shill number and related stats by dict lookup
parcels_w_sales['CaseShillMvt'] = parcels_w_sales['Year'].apply(lambda x: case_shill_mvt.get(x))
# free memory
del(case_shill_idx, case_shill_mvt)

CENSUS BUREAU

In [ ]:
# TLRESCONS: Total Res Construction Spending

# get the index
res_const = quandl.get("FRED/TLRESCONS", start_date=start_date, end_date=end_date) 
# group the years and get the sum of the differences for each year
res_const_mvt = getPeriodicIndexMovement(res_const)
# set the non res const value for each sale using a dict lookup
parcels_w_sales['ResConst'] = parcels_w_sales['Year'].apply(lambda x: res_const_mvt.get(x))
# free memory
del(res_const, res_const_mvt)

In [ ]:
# TLPRVCONS: total private construction spending

# get the index
tot_priv_const = quandl.get("FRED/TLPRVCONS", start_date=start_date, end_date=end_date) 
# group the years and get the sum of the differences for each year
tot_priv_const_mvt = getPeriodicIndexMovement(tot_priv_const)
# set value for each sale using a dict lookup
parcels_w_sales['PrivConst'] = parcels_w_sales['Year'].apply(lambda x: tot_priv_const_mvt.get(x))
# free memory
del(tot_priv_const, tot_priv_const_mvt)

In [ ]:
# WPUSI012011: construction price index

# get the index
con_price_idx = quandl.get("FRED/WPUSI012011", start_date=start_date, end_date=end_date)
# get index movement
con_price_idx_mvt = getPeriodicIndexMovement(con_price_idx)
# set value for each sale using a dict lookup
parcels_w_sales['ConstPriceIdx'] = parcels_w_sales['Year'].apply(lambda x: con_price_idx_mvt.get(x))
# free memory
del(con_price_idx, con_price_idx_mvt)

In [ ]:
# HIHAWA0URN: hi county jobless claims

# get the index
hi_cty_jobless = quandl.get("FRED/HIHAWA0URN", start_date=start_date, end_date=end_date)
# get index movement
hi_cty_jobless_mvt = getPeriodicIndexMovement(hi_cty_jobless)
# set value for each sale using a dict lookup
parcels_w_sales['HiCtyJobless'] = parcels_w_sales['Year'].apply(lambda x: hi_cty_jobless_mvt.get(x))
# free memory
del(hi_cty_jobless, hi_cty_jobless_mvt)

In [ ]:
# PCPI15001: per capita personal income, hawaii county

# get the index
hi_cty_percapinc = quandl.get("FRED/PCPI15001", start_date=start_date, end_date=end_date)
# get index movement
hi_cty_percapinc_mvt = getPeriodicIndexMovement(hi_cty_percapinc)
# set value for each sale using a dict lookup
parcels_w_sales['PerCapInc'] = parcels_w_sales['Year'].apply(lambda x: hi_cty_percapinc_mvt.get(x))
# free memory
del(hi_cty_percapinc, hi_cty_percapinc_mvt)

In [ ]:
# HSN1F: new one fam home sales, all US

# get the index
one_fam_sales = quandl.get("FRED/HSN1F", start_date=start_date, end_date=end_date)
# get index movement
one_fam_sales_mvt = getPeriodicIndexMovement(one_fam_sales)
# set value for each sale using a dict lookup
parcels_w_sales['1FamHomeSales'] = parcels_w_sales['Year'].apply(lambda x: one_fam_sales_mvt.get(x))
# free memory
del(one_fam_sales, one_fam_sales_mvt)

In [ ]:
# HOWNRATEACS015001: home ownership, hawaii county

# get the index
hi_cty_homeown = formatIndicatorLikeQuandl('HOWNRATEACS015001')
# get index movement
hi_cty_homeown_mvt = getAnnualIndexMovement(hi_cty_homeown)
# set value for each sale using a dict lookup
parcels_w_sales['HiCtyHomeown'] = parcels_w_sales['Year'].apply(lambda x: hi_cty_homeown_mvt.get(x))
# free memory
del(hi_cty_homeown, hi_cty_homeown_mvt)

FEDERAL HOUSING FINANCE AUTHORITY

In [ ]:
# USSTHPI: All-Transactions-House-Price-Index-for-the-United-States

# get the index
hpi_us = quandl.get("FRED/USSTHPI", start_date=start_date, end_date=end_date)
# get index movement
hpi_us_mvt = getPeriodicIndexMovement(hpi_us)
# set value for each sale using a dict lookup
parcels_w_sales['HPI US'] = parcels_w_sales['Year'].apply(lambda x: hpi_us_mvt.get(x))
# free memory
del(hpi_us, hpi_us_mvt)

In [ ]:
# HISTHPI: All-Transactions-House-Price-Index-for-Hawaii

# get the index
hpi_hi = quandl.get("FRED/HISTHPI", start_date=start_date, end_date=end_date)
# get index movement
hpi_hi_mvt = getPeriodicIndexMovement(hpi_hi)
# set value for each sale using a dict lookup
parcels_w_sales['HPI HI'] = parcels_w_sales['Year'].apply(lambda x: hpi_hi_mvt.get(x))
# free memory
del(hpi_hi, hpi_hi_mvt)

In [ ]:
# FIX30YR: Mortgage Interest Rate Index, US 

# get the index
mort_interest = quandl.get("FMAC/FIX30YR", start_date=start_date, end_date=end_date).loc[:,'US Interest Rate']
# get index movement
mort_interest_mvt = getPeriodicIndexMovement(pd.DataFrame(mort_interest).rename({'US Interest Rate':'Value'}, axis=1))
# set value for each sale using a dict lookup
parcels_w_sales['FredMacMortInt'] = parcels_w_sales['Year'].apply(lambda x: mort_interest_mvt.get(x))
# free memory
del(mort_interest, mort_interest_mvt)

In [ ]:
# remove the year col
parcels_w_sales.drop(['Year'], axis=1, inplace=True)

## Step 2: Inspect Data Distributions

#### Some Features

SLOPE MEAN 0-7

The box plot targets a geographical feature of the real estate, Slope Mean.

The conclusions are drawn on observations limtied to parcels with profitable sales records. 

The Slope Mean observations are divided into 50 bins (contiguous ranges). Each occurance of a bin is tabulated and the counts are graphed. In this case, the bins that occur 75% or more of the time have between 1261-1268 observations each, representing the entire range of values 0-20.

Properties with Slope Mean 0 <= x <= 7 should be favored. There are many observations clustered around 12-25.

In [ ]:
pd.qcut(parcels_w_sales[parcels_w_sales['Profitable Sale'] ==1]['SLOPE_MEAN'], q=50, duplicates='drop')\
    .value_counts()\
    .plot('box', figsize=(10,7))

ELEV MEAN: 50-400

The box plot targets a monetary feature of the real estate, Bldg Exempt.

The conclusions are drawn on observations limtied to parcels with profitable sales records. 

The Bldg Exempt observations are divided into 50 bins (contiguous ranges). Each occurance of a bin is tabulated and the counts are graphed. In this case, the bins that occur 75% or more of the time have between 1262-1265 observations each, representing nearly the entire range of values 0-3964.

Properties with Land Value 85 <= x <= 1000 should be favored with the main focus on those between 0-400. This suggests the market prefers property that is positioned at or slightly above the beach at elevations between 270-1300 feet.

In [ ]:
pd.qcut(parcels_w_sales[parcels_w_sales['Profitable Sale'] ==1]['ELEV_MEAN'], q=50, duplicates='drop')\
    .value_counts()\
    .plot('box', figsize=(10,7))

LAND VALUE: <=250K

The box plot targets a monetary feature of the land, Land Value.

The conclusions are drawn on observations limtied to parcels with profitable sales records. 

The Land Value observations are divided into 50 bins (contiguous ranges). Each occurance of a bin is tabulated and the counts are graphed. In this case, the bins that occur 75% or more of the time have between 1300-1500 observations each, representing nearly the entire range of values 100k-2M.

Properties with Land Value <=2M should be favored. The market favors those <=250k.

In [ ]:
pd.qcut(parcels_w_sales[parcels_w_sales['Profitable Sale'] == 1]['landvalue'], q=50, duplicates='drop')\
    .value_counts()\
    .plot('box', figsize=(10,7))

GISACRES: 1

The box plot targets a geographical feature of the land, GIS Acers.

The conclusions are drawn on observations limtied to parcels with profitable sales records.

The GIS Acres observations are divided into 50 bins (contiguous ranges). Each occurance of a bin is tabulated and the counts are graphed. In this case all bins have ~3000 observations each representing the entire range of values .01-2.7k.

Properties with GIS Acerage .2 <= x <=1.5 should be favored with special emphasis on around 1 acre of land. Note that there are very few sales where acerage exceeds 1. Few large parcels are ever sold.

In [ ]:
pd.cut(parcels_w_sales[parcels_w_sales['Profitable Sale'] == 1]['gisacres'], bins=4, duplicates='drop')\
    .value_counts()\
    .plot('box', figsize=(10,7))

TAX ACRES: .5-1

This box plot targets a geographcial feature of the real estate, Tax Acres.

The conclusions are drawn on observations limtied to parcels with profitable sales records. 

The Tax Acres observations are divided into 50 bins (contiguous ranges). Each occurance of a bin is tabulated and the counts are graphed. In this case, the bins that occur 75% or more of the time have between 1k-1.8k observations each, representing nearly the entire range of values .002 - 2750.

Properties with Tax Acres .1 <= x <= 1  should be favored.

In [ ]:
pd.cut(parcels_w_sales[parcels_w_sales['Profitable Sale'] == 1]['taxacres'], bins=3, duplicates='drop')\
    .value_counts()\
    .plot('box', figsize=(12,9))

TOT TAX VAL: 150k

This box plot targets a monetary feature of the real estate, Total Tax Val.

The conclusions are drawn on observations limtied to parcels with profitable sales records. 

The Total Tax Val observations are divided into 50 bins (contiguous ranges). Each occurance of a bin is tabulated and the counts are graphed. In this case, the bins that occur 75% or more of the time have between 1.2k-1.5k observations each, representing nearly the entire range of values 27k-32M .

Properties with Total Tax Val 100k <= x <= 250K should be favored. 

In [ ]:
pd.qcut(parcels_w_sales[parcels_w_sales['Profitable Sale'] ==1]['tot_tax_val'], q=50, duplicates='drop')\
    .value_counts()\
    .plot('box', figsize=(12,9))

MKT BLD VAL: $150k-$200k

This box plot targets a monetary feature of the real estate, Mkt Bld Val.

The conclusions are drawn on observations limtied to parcels with profitable sales records. 

The Mkt Bld Val observations are divided into 50 bins (contiguous ranges). Each occurance of a bin is tabulated and the counts are graphed. In this case, one bin (<=466k) holds the majority of observations, representing nearly the entire range of values which consists of values <=$500k.

Properties with Mkt Bld Val 125k <= x <=225K should be favored.

In [ ]:
pd.qcut(parcels_w_sales[parcels_w_sales['Profitable Sale'] ==1]['mkt_land_val'], q=50, duplicates='drop')\
    .value_counts()\
    .plot('box', figsize=(12,9))

CASE SHILL MVT: 2-12

The box plot targets an index feature of the economic state, The Case Shill Mvt.

The conclusions are drawn on observations limtied to parcels with profitable sales records. 

The Case Shill Mvt observations are divided into 50 bins (contiguous ranges). Each occurance of a bin is tabulated and the counts are graphed. In this case, the bins that occur 75% or more of the time have between 3k-5k observations each, representing nearly the entire range of values 2-12.

Case Shill Mvt of 2 <= x <= 12 should be favored with lulls between 12-15 and a spike between 15-20, possibly signaling hesitation as the market climbs and extatic behavior at the height of the housing market.

There is a spike in sales at the 2-3 interval, A bulk of sales at the 8-10, and another spike at 11-12. Sales jump rather than plod.

Overall, positive Case Shill Mvt signals more sales.

In [ ]:
pd.qcut(parcels_w_sales[parcels_w_sales['Profitable Sale'] == 1]['CaseShillMvt'], q=50, duplicates='drop')\
    .value_counts()\
    .plot('box', figsize=(12,9))

CONST PRICE IDX: 2-5

The box plot targets an index feature of the economic state, The Const Price Idx Rate.

The conclusions are drawn on observations limtied to parcels with profitable sales records. 

The Const Price Idx observations are divided into 50 bins (contiguous ranges). Each occurance of a bin is tabulated and the counts are graphed. In this case, the bins that occur 75% or more of the time have between 2.5k - 5.5k observations each, representing nearly the entire range of values 1.4 - 5.3.

Const Price Idx rates 0 <= x <= 5 should be favored, with a small spike around 15. Within the range, there are spikes at precisely 2-3, 4, and 5. This suggests jumps in sales rather than plodding increases. It could also be an effect of monthly measurement procedures. An interpolated value would smooth, accounting for procedure.

In [ ]:
pd.qcut(parcels_w_sales[parcels_w_sales['Profitable Sale'] == 1]['ConstPriceIdx'], q=50, duplicates='drop')\
    .value_counts()\
    .plot('box', figsize=(12,9))

1 FAM HOME SALES: ~50

The box plot targets an index feature of the economic state, The Single Family Home Sales Rate.

The conclusions are drawn on observations limtied to parcels with profitable sales records. 

The Single Family Home Sales observations are divided into 50 bins (contiguous ranges). Each occurance of a bin is tabulated and the counts are graphed. In this case, the bins that occur 75% or more of the time have between 3k-6.5k observations each, representing nearly the entire range of values -19 to 77.

Single Family Home Sales rates 0 <= x <= 100 should be favored. A sharp spike occurs at around 50. This shows that the Hawaiian housing market is positively coorelated to the broader economy of home sales. When the home sales rate increases, profitability increases. Thus, the Hawaiian market is predominately comprised of single-family home sales.

In [ ]:
pd.qcut(parcels_w_sales[parcels_w_sales['Profitable Sale'] == 1]['1FamHomeSales'], q=50, duplicates='drop')\
    .value_counts()\
    .plot('box', figsize=(12,9))

FRED MAC MORT INT: -.5 to .5

The box plot targets an index feature of the economic state, The Fred Mac Mort Int Rate.

The conclusions are drawn on observations limtied to parcels with profitable sales records. 

The Fred Mac Mort Int observations are divided into 50 bins (contiguous ranges). Each occurance of a bin is tabulated and the counts are graphed. In this case, the bins that occur 75% or more of the time have between 3.5k-5k observations each, representing nearly the entire range of values -.66 to .3.

Fred Mac Mort Int rates -.5 <= x <= .5 should be favored. The market favors stability in the mortgage interest rate, with a slight bias towards downward momentum.

In [ ]:
pd.qcut(parcels_w_sales[parcels_w_sales['Profitable Sale'] == 1]['FredMacMortInt'], q=50, duplicates='drop')\
    .value_counts()\
    .plot('box', figsize=(12,9))

## Step 3: Create Machine Learning Model (Supervised)

#### Load dependencies

In [ ]:
import scipy as sp
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.metrics import recall_score, precision_score, precision_recall_curve, roc_curve, auc

#### Prep backtest data: Cleanup Data

In [ ]:
# remove buff dist from geometry
parcels_w_sales.drop('buff_dist', axis=1, inplace=True)
# set geometry back to unbuffered polygons
parcels_w_sales = parcels_w_sales.set_geometry('geometry')
# set homeowner to binary
parcels_w_sales['homeowner'] = parcels_w_sales['homeowner'].replace({'N':0, 'Y':1, 'U': np.NaN})

In [ ]:
# reference unnecessary cols and prevent data leakage
cols_to_drop =['st_areashape', 'district', 'geometry']
# drop cols
parcels_w_sales.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
# change the property class to a numeric value with a lookup
prop_class_lookup = {}
# iterate through prop class vals and assign to dict
for idx, i in enumerate(parcels_w_sales['property_class'].unique()):
    prop_class_lookup[i] = idx
# assign key lookup to property class col
parcels_w_sales['property_class'] = parcels_w_sales['property_class'].apply(lambda x: prop_class_lookup.get(x))

In [ ]:
# get counts of property class to compare
parcels_w_sales[parcels_w_sales['Profitable Sale']==1]['property_class'].value_counts().plot('bar');
for key, val in prop_class_lookup.items():
    print(val, ':', key)

In [ ]:
# make a clean index
parcels_w_sales.reset_index(drop=True, inplace=True)

In [ ]:
# convert Sale Intvl to years
parcels_w_sales['Sale Intvl'] = parcels_w_sales['Sale Intvl'] / pd.Timedelta(365, 'D')

#### Prep Current Data

In [ ]:
import arcgis

In [ ]:
# remake the parcels link
parcels_link = ''.join([HI_geo_stem, HI_parcels_json, '.geojson'])
# load hi parcels as arcgis geodataframe 
hi_arc_parcels = arcgis.features.FeatureSet.from_geojson(requests.get(parcels_link).json()).df
# remove qpub link from parcels
hi_arc_parcels['qpub_link'] = hi_arc_parcels['qpub_link'].apply(lambda x: x[71:]).astype(int)
hi_arc_parcels.rename({'qpub_link':'parcel'}, inplace=True, axis=1)
# select only features supplied to classifier 
columns = ['bldgexempt', 'bldgvalue', 'gisacres',
           'homeowner', 'landexempt', 'landvalue', 
           'pittcode', 'parcel', 
           'taxacres', 'SHAPE']
hi_arc_parcels = hi_arc_parcels[columns]
# convert dtypes for cols as necessary
hi_arc_parcels['homeowner'] = hi_arc_parcels['homeowner'].replace({'N':0, 'Y':1, 'U': np.NaN})
# add a Sale Date col to hi_arc_parcels
hi_arc_parcels['Sale Date'] = pd.to_datetime('today')

In [ ]:
## limit parcels to only the most recent record

# limit columns in parcels_w_sales 
cols_to_drop = [
    'bldgexempt', 'bldgvalue', 'gisacres', 'homeowner',
    'landexempt', 'landvalue', 'pittcode', 'taxacres',
    'Sale Amount', 'Profitable Sale', 'Sale Intvl',
    
    'Fed Funds', 'CaseShillMvt', 'ResConst', 'PrivConst', 
    'ConstPriceIdx', 'HiCtyJobless', 'PerCapInc', '1FamHomeSales', 
    'HiCtyHomeown', 'HPI US', 'HPI HI', 'FredMacMortInt'
]
# container to hold latest_sale history
latest_sales = []
# create an index of parcels from existing sales histroy 
to_do = parcels_w_sales['parcel'].unique().tolist()
# sort all parcels by date
parcels_w_sales.sort_values(['Sale Date'], ascending=False, inplace=True)

# loop over sorted parcels with needed cols only
for parcel in parcels_w_sales.drop(cols_to_drop, axis=1).iterrows():
    # if parcel in remaining parcels
    if parcel[1].loc['parcel'] in to_do:
        # add this most recent record to stack
        latest_sales.append(parcel[1])
        # pop off of remaining parcels  
        to_do.remove(parcel[1].loc['parcel'])

# make dataframe from most recent parcels
latest_sales = pd.DataFrame(latest_sales)
# concat with hi_arc_parcels by calculating sale intvl
final_sales = pd.concat([latest_sales, hi_arc_parcels])

In [ ]:
# isolate only parcels where there are 2 records, one old, one new
final_sales = final_sales[final_sales['parcel'].isin((final_sales['parcel'].value_counts() == 2).replace({False:np.NaN}).dropna().index)]
# reset index
final_sales.reset_index(inplace=True, drop=True)
# sort the frame by parcel then Sale Date
final_sales.sort_values(['parcel', 'Sale Date'], ascending=False, inplace=True)
# calculate sale interval
final_sales['Sale Intvl'] = final_sales['Sale Date'].diff(-1)
# merge every 2nd record to get all records together: merge oldest with newest
final_sales = pd.merge(final_sales.iloc[::2], final_sales.iloc[1::2], on='parcel')

In [ ]:
# remove cols with NAN
cols_to_drop = [
    'ELEV_MAX_x', 
    'ELEV_MEAN_x',
    'ELEV_MIN_x',
    'ELEV_RANGE_x',
    'ELEV_STD_x',
    'SLOPE_MAX_x',
    'SLOPE_MEAN_x', 
    'SLOPE_MIN_x', 
    'SLOPE_RANGE_x', 
    'SLOPE_STD_x',
    
    'roads_nearby_x', 
    'schools_nearby_x', 
    'SHAPE_y', 
    'Sale Date_y', 
    'bldgexempt_y', 
    'bldgvalue_y',
    'gisacres_y', 
    'homeowner_y', 
    'landexempt_y', 
    'landvalue_y', 
    'mkt_bld_val_x',
    'mkt_land_val_x', 
    'pittcode_y', 
    'property_class_x', 
    'taxacres_y',
    'tot_tax_val_x', 
    'Sale Intvl_y', # x | y
    'zoning_id_x', 
    'land_area_x'
]
final_sales.drop(cols_to_drop, inplace=True, axis=1)
# remove suffixes 
final_sales.columns = final_sales.columns.str.replace('(_y)|(_x)', '')
# convert Sale Intvl to years
final_sales['Sale Intvl'] = final_sales['Sale Intvl'].dt.days / 365
# merge hi_arc_parcels with final_sales to convert to spatial df
hi_arc_parcels = hi_arc_parcels.merge(final_sales, on='parcel')
# free memory
del(final_sales, columns, parcel)

In [ ]:
# remove unneeded cols by suffixes 
hi_arc_parcels.drop([col for col in hi_arc_parcels.columns if '_x' in col], inplace=True, axis=1)
# remove suffixes in required cols
hi_arc_parcels.columns = hi_arc_parcels.columns.str.replace('(_x)|(_y)', '')

In [ ]:
# create dict of present-day values
present_day_indexes = {
    'Fed Funds':getRecentIdxMovement('FRED/DFF', 'quandl'),
    'CaseShillMvt':getRecentIdxMovement('CSUSHPINSA', 'fred'),
    'ResConst':getRecentIdxMovement('FRED/TLRESCONS', 'quandl'),
    'PrivConst':getRecentIdxMovement('FRED/TLPRVCONS', 'quandl'),
    'ConstPriceIdx':getRecentIdxMovement('FRED/WPUSI012011', 'quandl'),
    'HiCtyJobless':getRecentIdxMovement('FRED/HIHAWA0URN', 'quandl'),
    'PerCapInc':getRecentIdxMovement('FRED/PCPI15001', 'quandl'),
    'HiCtyHomeown':getRecentIdxMovement('HOWNRATEACS015001', 'fred'),
    '1FamHomeSales':getRecentIdxMovement('FRED/HSN1F', 'quandl'),
    'HPI US':getRecentIdxMovement('FRED/USSTHPI', 'quandl'),
    'HPI HI':getRecentIdxMovement('FRED/HISTHPI', 'quandl'),
    'FredMacMortInt':getRecentIdxMovement('FMAC/FIX30YR', 'quandl'),
    }

# broadcast values to present_day parcels
for key in present_day_indexes.keys():
    hi_arc_parcels[key] = present_day_indexes[key]

## Run ML Model and Score

#### Train Test backtest data splits 

In [ ]:
X = parcels_w_sales[[i for i in parcels_w_sales.columns if i != 'Profitable Sale']]
y = parcels_w_sales['Profitable Sale'] 
# fillna() for decision tree 
X.fillna(-99999999.0, inplace=True)
# make train / test splits
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, shuffle=True)

In [ ]:
# free memory
del(parcels_w_sales, latest_sales)

#### Instantiate classifier and fit with data

In [ ]:
# instantiate a Decision Tree classifier to capture feature importances
clf = DecisionTreeClassifier(max_depth=10, random_state=10) 

In [ ]:
# fit the training data, ignoring geometry, parcel, and Sale Date cols to prevent leakage
mask =[i for i in Xtrain.columns if i != 'geometry' and i != 'parcel' and i != 'Sale Date' and i != 'Sale Amount']
clf.fit(Xtrain[mask], ytrain)

#### Score the model

In [ ]:
# score the classifier, ignoring geometry, parcel, and Sale Date cols
clf.score(Xtest[mask], ytest)

In [ ]:
# capture backtested predictions for scoring 
predictions = clf.predict(Xtest[mask])
# capture accuracy and recall scores
accuracy = np.round(clf.score(Xtest[mask], ytest) * 100, 2); print(accuracy)
recall = np.round(recall_score(ytest, predictions) * 100, 2) ; print(recall)

In [ ]:
# compute the roc curve
fpr, tpr, thresholds = roc_curve(ytest, predictions)
# compute the area under receiver operator
auc(fpr, tpr)

#### Show feature importances
These were the 10 most important features for predicting profitability.

In [ ]:
# get feature importances using indices to get column names
sorted_feats = Xtest.columns[np.argsort(clf.feature_importances_)[-10:].tolist()]
sorted_scores = np.sort(clf.feature_importances_)[-10:]
# iterate through sorted values and print feature importance name and score
for feat, score in zip(sorted_feats, sorted_scores):
    print(feat+':', np.round(score, 3))

#### Run on Current Data

In [ ]:
# fillna() for decision tree
hi_arc_parcels.fillna(.000000000000001, inplace=True)

In [ ]:
arc_mask = [ 'bldgexempt', 'bldgvalue', 'gisacres',
       'homeowner', 'landexempt', 'landvalue', 'pittcode', 'taxacres',
       'Sale Intvl', 'ELEV_MAX', 'ELEV_MEAN',
       'ELEV_MIN', 'ELEV_RANGE', 'ELEV_STD', 'SLOPE_MAX', 'SLOPE_MEAN',
       'SLOPE_MIN', 'SLOPE_RANGE', 'SLOPE_STD', 'land_area', 'mkt_bld_val',
       'mkt_land_val', 'property_class', 'roads_nearby', 'schools_nearby',
       'tot_tax_val', 'zoning_id', 'Fed Funds', 'CaseShillMvt', 'ResConst',
       'PrivConst', 'ConstPriceIdx', 'HiCtyJobless', 'PerCapInc',
       'HiCtyHomeown', '1FamHomeSales', 'HPI US', 'HPI HI', 'FredMacMortInt']

In [ ]:
# predict_proba on current data
hi_arc_parcels['prediction'] = clf.predict_proba(hi_arc_parcels[[col for col in hi_arc_parcels.columns if col != 'SHAPE' and col != 'parcel' and col != 'Sale Date' and col != 'Sale Amount']])[:,1]

In [ ]:
# convert parcel to string for display
hi_arc_parcels['parcel'] = hi_arc_parcels['parcel'].astype(str)

In [ ]:
# limit display selection 
hi_arc_parcels = hi_arc_parcels[['SHAPE', 'parcel', 'prediction']]

## Step 5: Visualize Results with ArcGIS

### Dependencies

In [ ]:
from IPython.display import display
import arcgis

#### Instantiate an ArcGIS instance 

In [ ]:
# get a gis
gis = arcgis.GIS("https://python.playground.esri.com/portal", "arcgis_python", "amazing_arcgis_123")

#### Backtested results in a map

In [ ]:
# create a map widget and center on Hilo, Hawaii
backtest_map = gis.map('Hilo, Hawaii', zoomlevel=9)
backtest_map.basemap = 'gray'

In [ ]:
# predict that the observed data is True for profitability and assign predict_true to Xtest
Xtest['prediction'] = clf.predict_proba(Xtest[mask])[:,1]
# recombine train and test data
Xtest['actual'] = ytest
# get only desired cols for display
cols = ['parcel', 'Sale Date', 'prediction', 'actual']
# convert Sale Date to string for display
Xtest['Sale Date'] = Xtest['Sale Date'].astype(str)
# set cols
Xtest = Xtest[cols]

In [ ]:
# retrieve latest parcel shapes one more time
parcels_link = ''.join([HI_geo_stem, HI_parcels_json, '.geojson'])
# load hi parcels as arcgis geodataframe 
arc_parcels = arcgis.features.FeatureSet.from_geojson(requests.get(parcels_link).json()).df
# remove qpub link from parcels
arc_parcels['qpub_link'] = arc_parcels['qpub_link'].apply(lambda x: x[71:]).astype(int)
arc_parcels.rename({'qpub_link':'parcel'}, inplace=True, axis=1)
# select only SHAPE col 
arc_parcels = arc_parcels[['parcel','SHAPE']]
# convert to arcgis geodataframe, merging SHAPEs on parcel 
Xtest = arc_parcels.merge(Xtest, on='parcel', how='inner').drop('parcel', axis=1)

In [ ]:
# add the layer to the map
backtest_map.add_layer(Xtest.to_featureset(), {"renderer":"ClassedColorRenderer", "field_name":"prediction"})
backtest_map

#### Current parcel predictions in a map

In [ ]:
# create a map widget and center on Hilo, Hawaii
current_map = gis.map('Hilo, Hawaii', zoomlevel=9)
current_map.basemap = 'gray'

In [ ]:
# show the counts of predictions
hi_arc_parcels['prediction'].value_counts()

In [ ]:
# add first 200 records where prediction is low (<=50%)
current_map.add_layer(hi_arc_parcels[hi_arc_parcels['prediction'] <=.5].sort_values(['prediction'], ascending=False).iloc[:200].to_featureset(), {"renderer":"ClassedColorRenderer", "field_name":"prediction"})

In [ ]:
# view the 200 lowest predictions ...this is where investors might find bargain opportunities
current_map